# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [57]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.dot(data_set, data_set.T) ** d
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [58]:
def choose_set_for_label(data_set, labels, classes):
    indexes = np.where(np.logical_or(labels == classes[0], labels == classes[1]))[0]
    choosen_labels = labels[indexes]
    train_labels = np.where(choosen_labels == classes[0], -1, 1)
    train_data_set = data_set[indexes]
    return train_data_set, train_labels

In [59]:
def get_labels_count(data_set):
    labels_count = len(np.unique(data_set))
    return labels_count

Use the code that we have implemented earlier:

In [60]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_data_set)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [61]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from itertools import combinations

iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, labels, test_size=0.4, random_state=15)

predictions = []
labels_combinations = list(combinations(range(get_labels_count(labels)), 2))

for combination in labels_combinations:
    train_data, train_labels_new = choose_set_for_label(train_data_set, train_labels, combination)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data, train_labels_new, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = np.where(predicted == -1, combination[0], combination[1])
    predictions.append(predicted.flatten())

predictions = np.array(predictions)

predicted = []
for i in range(len(test_data_set)):
    classified = predictions[:, i]
    predicted.append(np.argmax(np.bincount(classified)))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.4671e+01 -9.7399e+02  1e+03  7e-16  2e-15
 1:  1.3716e+01 -4.5874e+01  6e+01  7e-16  3e-15
 2:  8.9814e-02 -5.6899e+00  6e+00  3e-16  1e-15
 3: -1.2866e+00 -1.9302e+00  6e-01  2e-16  5e-16
 4: -1.4097e+00 -1.5815e+00  2e-01  2e-16  3e-16
 5: -1.4742e+00 -1.5285e+00  5e-02  3e-16  3e-16
 6: -1.4996e+00 -1.5107e+00  1e-02  2e-16  2e-16
 7: -1.5058e+00 -1.5060e+00  2e-04  8e-16  3e-16
 8: -1.5059e+00 -1.5059e+00  3e-06  3e-16  4e-16
 9: -1.5059e+00 -1.5059e+00  3e-08  2e-16  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0473e+02 -1.0015e+03  1e+03  9e-16  2e-15
 1:  1.5411e+01 -5.1601e+01  7e+01  5e-16  3e-15
 2: -5.8770e-03 -6.5701e+00  7e+00  2e-16  1e-15
 3: -1.5724e+00 -2.2854e+00  7e-01  2e-16  4e-16
 4: -1.7594e+00 -1.9425e+00  2e-01  1e-16  2e-16
 5: -1.8170e+00 -1.8703e+00  5e-02  4e-16  3e-16
 6: -1.8430e+00 -1.8563e+00  1e-02  7e-16  3e-16
 7: -1.8478e+00 -1.8484e+00  7e-04  3e-16  3e-1